# SnapPy and SageMath are friends!

* SnapPy always uses part of Sage, namely its interface to PARI, repackaged into the stand-alone module CyPari:

    - Smith normal form for homology computations.

    - Arbitrary precision arithmetic.

    - Available on PyPI.
    

* When installed into Sage, via:
```
sage -pip install --no-use-wheel snappy
```
SnapPy gains additional functionality.  GUI still works:
```
sage -python -m snappy.app
```


* Installed on SageMathCloud.
   

## Sage-specific features.

Numeric return types are native Sage types.

In [1]:
%gui tk
import snappy
M = snappy.Manifold('m004')
type(M.volume())

sage.rings.real_mpfr.RealNumber

In [2]:
MH = M.high_precision()
%time MH.volume()

CPU times: user 7.48 ms, sys: 1.13 ms, total: 8.61 ms
Wall time: 15.2 ms


2.02988321281930725004240510854904057188337861506059958403497821

In [3]:
vol = MH.volume()
vol.parent()

Real Field with 212 bits of precision

In [4]:
M.tetrahedra_shapes('rect', bits_prec=1000)[0]


0.500000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 + 0.866025403784438646763723170752936183471402626905190314027903489725966508454400018540573093378624287837813070707703351514984972547499476239405827756047186824264046615951152791033987410050542337461632507656171633451661443325336127334460918985613523565830183930794009524993268689929694733825173753288025*I

The quad-double precision kernel allows computing very complicated Dirichlet domains.  



In [5]:
B = snappy.Manifold('13a100')
print(B.volume())
try:
    B.dirichlet_domain()
except RuntimeError, error:
    print(error)

15.2039621209048
The Dirichlet construction failed.


In [6]:
BH = B.high_precision()
D = BH.dirichlet_domain(); D
B.plink(), D.view()

Starting the link editor.
Select Tools->Send to SnapPy to load the link complement.


(None, None)

Because of local rigidity, the shapes are always algebraic numbers.  We can use LLL to recover their exact expressions, following Goodman, Neumann, et. al.

In [7]:
TF = M.tetrahedra_field_gens()
TF

<SetOfAAN: [0.5 + 0.8660254037844386*I, 0.5 + 0.8660254037844386*I]>

In [8]:
TF.find_field(100, 10, True)

(Number Field in z with defining polynomial x^2 - x + 1,
 <ApproxAN: 0.5 + 0.866025403784*I>,
 [z, z])

Finite covers are of central interest in 3-manifold theory, e.g. Agol's recent solution to the Virtual Haken Conjecture. SnapPy has some basic ability here:

In [9]:
%time len(M.covers(9))

CPU times: user 3.66 s, sys: 11.3 ms, total: 3.67 s
Wall time: 3.67 s


11

However, in Sage one can use GAP or Magma to explore *much* bigger covers.

In [10]:
from sage.all import gap, magma, PSL
gap(1), magma(1) # one-time startup cost
%time covers = M.covers(9, method='gap')

CPU times: user 213 ms, sys: 30.2 ms, total: 243 ms
Wall time: 284 ms


In [11]:
%time covers = M.covers(9, method='magma')

CPU times: user 55.1 ms, sys: 11.6 ms, total: 66.7 ms
Wall time: 224 ms


In [12]:
[C.homology() for C in covers]

[Z/21 + Z,
 Z + Z + Z,
 Z + Z + Z,
 Z + Z + Z,
 Z/21 + Z,
 Z/4 + Z/4 + Z,
 Z/3 + Z + Z + Z,
 Z/3 + Z + Z + Z,
 Z/3 + Z + Z + Z,
 Z/3 + Z + Z + Z,
 Z/76 + Z/76 + Z]

In [13]:
G = M.fundamental_group(); G

Generators:
   a,b
Relators:
   aaabABBAb

In [14]:
G_gap = gap(G)
G_gap.RelatorsOfFpGroup()

[ a^3*b*a^-1*b^-2*a^-1*b ]

In [15]:
f = G_gap.GQuotients(PSL(2,7))[1]
C = M.cover(f.Kernel())
C.volume()/M.volume()

167.999999999999

In [16]:
G_mag = magma(G)
H = G_mag.LowIndexSubgroups(12)[174]
N = G_mag.Core(H)
print(G_mag.Index(N))
H = M.cover(N).homology()
from sage.all import AbelianGroup
AbelianGroup(H.elementary_divisors())

576


Multiplicative Abelian group isomorphic to C2 x C4 x C4 x C4 x C4 x C20 x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z x Z

## Verified computation

As pioneered by HIKMOT, SnapPy can use interval arithmetic to *prove* that a given manifold is hyperbolic and provide intervals where the exact shapes must lie.  Uses Sage's complex interval types and the Newton interval method.

In [17]:
E = snappy.Manifold('K14n1234')
success, shapes = E.verify_hyperbolicity()
success, shapes[:5]

(True,
 [1.171146365? + 1.625447436?*I,
  0.1105132481? + 0.7195638777?*I,
  0.5400664982? + 1.0871004030?*I,
  1.097382711? + 0.886360266?*I,
  0.302481620? + 0.8770938905?*I])

In [18]:
z0 = shapes[0]
z0.diameter()

3.24484363906224e-10

In [19]:
better_shapes = E.verify_hyperbolicity(bits_prec=1000)[1]
max(z.diameter() for z in better_shapes)

6.53171489530889204000291398071224051868662363632504549344497621627402116334874914750325382397429079390135189615865916625036497160206157552137518606107459293503610749359326584929731245435811280206005757224925183922994477338855957052162232575043174260030219734392466310677413445991369115552977045218972e-295

By Mostow rigidity, a (finite-volume) hyperbolic structure is unique.  When the manifold has cusps, there is a canonical ideal cell decomposition associated to the hyperbolic structure.  SnapPy uses this to decide when two hyperbolic manifolds are homeomorphic.  

In [20]:
F = snappy.Manifold('K14n1235')
E.is_isometric_to(F)

False

That was a numerical calculation and not rigorous because SnapPy might have miscalculated the canonical decompositions.  Let's fix that.

In [21]:
E.num_tetrahedra(), E.triangulation_isosig()

(17, 'rLLvLvALQMQQcceiklpnmpmmoqnoqoqiidaihuaddexnckngk_bBrtRs')

In [22]:
E.isometry_signature(verified=True)

'AvLLLMvLLvzAQMMQQQccdgjhimrswsoxtvuyzrwxuyzwxzgfaagfegjlgcxokkpdbamgpmpmg'

In [23]:
F.isometry_signature(verified=True)

'ALLzvLzzQvzPQLMQPQcccdkinmjoputqustxwywvvxyzzzqffaaaaofgaamatacwargggqtpr'

**Note:** The latter two strings encode the *homeomorphism type* of these manifolds.

## Link Diagrams: Spherogram

Spherogram is a separately installable module, mostly pure-Python, which deals with knot and link diagrams.  Basic data structure is a planar diagram.

In [24]:
import spherogram
K = spherogram.random_link(300, 1, consistent_twist_regions=True)
K
K.view()

<plink.editor.LinkEditor instance at 0x497089680>

In [25]:
K_orig = K.copy()
K.simplify('global')
K, K_orig

(<Link: 1 comp; 51 cross>, <Link: 1 comp; 68 cross>)

In [26]:
K, K_orig

(<Link: 1 comp; 51 cross>, <Link: 1 comp; 68 cross>)

In [27]:
K.view()

<plink.editor.LinkEditor instance at 0x497b1b3f8>

In [28]:
vol = K.exterior().volume()
vol, vol/3.667

(81.1912781926650, 22.1410630468135)

In [29]:
K.alexander_polynomial()  # 'local' algorithm of Bar-Natan

48*t^26 - 944*t^25 + 9108*t^24 - 57412*t^23 + 265626*t^22 - 960719*t^21 + 2824548*t^20 - 6929507*t^19 + 14449840*t^18 - 25955924*t^17 + 40558062*t^16 - 55520974*t^15 + 66906792*t^14 - 71177087*t^13 + 66906792*t^12 - 55520974*t^11 + 40558062*t^10 - 25955924*t^9 + 14449840*t^8 - 6929507*t^7 + 2824548*t^6 - 960719*t^5 + 265626*t^4 - 57412*t^3 + 9108*t^2 - 944*t + 48

In [30]:
L = spherogram.Link('L13n131')

In [31]:
L.jones_polynomial()

-q^-2 + 2*q^-1 - 3 + 4*q - 6*q^2 + 5*q^3 - 6*q^4 + 4*q^5 - 4*q^6 + 3*q^7 - q^8 + q^9

In [32]:
L.signature()

5

In [33]:
L.seifert_matrix()

[ 0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 1 -1  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0 -1  0  0  0  0  0  0  0  0  0]
[-1  1  0  1 -1  0  0  0  0  0  0  0  0]
[ 0 -1  0  0  1 -1  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  1 -1  0  0  0  0  0]
[ 0  0 -1  0  0  1  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  1  0 -1  0  0  0]
[ 0  0  0  0  0  0  0  0  0  1 -1  0  0]
[ 0  0  0  0  0  0  0  0  0  0  1 -1  0]
[ 0  0  0  0  0 -1  0  0  0  0  0  1  0]
[ 0  0  0  0  0  0 -1  0  1  0  0  0  0]

In [34]:
D = L.morse_diagram()  # Uses Sage's interface to GLPK

In [35]:
print(D.is_bridge())
B = D.bridge()
len(B.crossings)

True


13

In [36]:
L.braid_word()

[-1, 2, -3, 4, -3, -2, 1, -2, 1, -2, 3, -4, -3, -3, -3, 2, -3]

## Spherogram 1.5 and SageMath 7.2 links and braids are friends

In [37]:
L_sage = L.sage_link()
type(L_sage)

sage.knots.link.Link

In [38]:
L_sage_and_back = spherogram.Link(L_sage)

L_sage_and_back.exterior().is_isometric_to(L.exterior())

True

In [39]:
w = L.braid_word(as_sage_braid=True)
w, w.parent()

(s0^-1*s1*s2^-1*s3*s2^-1*(s1^-1*s0)^2*s1^-1*s2*s3^-1*s2^-3*s1*s2^-1,
 Braid group on 5 strands)

In [40]:
L_braid = spherogram.Link(braid_closure=w)
L_braid

<Link: 2 comp; 17 cross>

In [41]:
L_braid.simplify('global')
L

<Link L13n131: 2 comp; 13 cross>

# Other 3-manifold software.

### Regina

Regina focuses on the purely topological side of things, especially normal surface theory.  Includes some support for higher-dimensional manifolds. Like SnapPy, it has a stand-alone GUI and also a Python interface. 

See http://unhyperbolic.org/sageRegina/ for how to install it painlessly into Sage.

### t3m

Light-weight pure Python library for dealing with triangulations of 3-manifolds.  Included with SnapPy as `snappy.snap.t3mlite`. 

# Possible projects for Sage Days 74

### Make SnapPy (even) easier to install into SageMath

   - Sage optional package?

   - SageMath binaries for OS X < 10.11? 

### Sage's "attach" blocks Tkinter 

In Sage's IPython-based interpreter, the mechanism behind Sage's attach functionality blocks IPython's ability to integrate with Tk (and other GUI's) event loops. (See trac #15152).



### Modernize CyPari

CyPari is based on Sage circa 2012, and there have been improvements since, with more to come (Demeyer et. al.).

### Modularization

Modularize some parts of the SageMath kernel, for example interval arithmetic, for use in stand-alone SnapPy.